# Importando Bibliotecas

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import numpy as np
import scipy

import plotly.graph_objects as go

from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from mlxtend.feature_selection import SequentialFeatureSelector

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

import itertools

import warnings
warnings.filterwarnings('ignore')

/Users/thiagoosorio/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Funções

In [2]:
def shapiro_test(data, alpha=0.05):
    stat, p = scipy.stats.shapiro(data)
    print('p-valor: p=%.3f' % (p))
    alpha = alpha
    if p > alpha:
        print('A amostra parece ser normalmente distribuída Gaussian. Não rejeitamos H0')
    else:
        print('A amostra NÃO parece ser normalmente distribuiída Gaussian. Rejeitamos H0')
    return stat, p

# Lendo os Dados

In [3]:
df = pd.read_csv('../Dados/df_final.csv')
df.head()

,data,vendas,vendas_Outra_Faculdade,vendas_PUCRIO,vendas_UFABC,vendas_UFF,vendas_UFMG,vendas_UFPE,vendas_UFRJ,vendas_UFSC,...,vendas_Outra_Cidade,vendas_Porto_Alegre,vendas_Recife,vendas_Rio_de_Janeiro,vendas_Salvador,vendas_Santo_Andre,vendas_Sao_Carlos,vendas_Sao_Paulo,trend,seasonal
0,2018-01-01,1819.00000,618.00000,100.00000,17.00000,194.00000,34.00000,71.00000,407.00000,23.00000,...,587.00000,11.00000,51.00000,521.00000,25.00000,9.00000,14.00000,187.00000,NaN,-2401.78443
1,2018-02-01,2012.00000,652.00000,98.00000,22.00000,199.00000,34.00000,85.00000,492.00000,28.00000,...,654.00000,11.00000,62.00000,592.00000,23.00000,13.00000,14.00000,205.00000,NaN,-1835.22888
2,2018-03-01,4035.00000,926.00000,229.00000,16.00000,357.00000,46.00000,163.00000,1473.00000,30.00000,...,1103.00000,14.00000,123.00000,1544.00000,22.00000,12.00000,29.00000,282.00000,NaN,935.86834
3,2018-04-01,4305.00000,1204.00000,189.00000,186.00000,512.00000,96.00000,134.00000,1098.00000,151.00000,...,1299.00000,21.00000,105.00000,1300.00000,38.00000,40.00000,27.00000,359.00000,NaN,670.79890
4,2018-05-01,3956.00000,1462.00000,176.00000,130.00000,349.00000,212.00000,158.00000,721.00000,122.00000,...,1250.00000,20.00000,110.00000,1065.00000,41.00000,39.00000,31.00000,300.00000,NaN,267.66001


---
# Time Series

## Prophet

In [4]:
df_prophet = df[['data', 'vendas']]
df_prophet.rename(columns={
    'data': 'ds',
    'vendas': 'y'
}, inplace=True)

In [5]:
model = Prophet()
model.fit(df_prophet)

15:51:44 - cmdstanpy - INFO - Chain [1] start processing
15:51:44 - cmdstanpy - INFO - Chain [1] done processing


In [6]:
forecast = model.predict(df_prophet)
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-01-01,3032.78718,-374.68029,1825.03558,3032.78718,3032.78718,-2302.05849,-2302.05849,-2302.05849,-2302.05849,-2302.05849,-2302.05849,0.00000,0.00000,0.00000,730.72870
1,2018-02-01,3198.97342,386.21988,2631.04814,3198.97342,3198.97342,-1617.85729,-1617.85729,-1617.85729,-1617.85729,-1617.85729,-1617.85729,0.00000,0.00000,0.00000,1581.11613
2,2018-03-01,3349.07712,2539.54397,4743.07994,3349.07712,3349.07712,342.37335,342.37335,342.37335,342.37335,342.37335,342.37335,0.00000,0.00000,0.00000,3691.45047
3,2018-04-01,3515.26336,2854.35978,5097.56419,3515.26336,3515.26336,491.26316,491.26316,491.26316,491.26316,491.26316,491.26316,0.00000,0.00000,0.00000,4006.52652
4,2018-05-01,3676.08875,3235.06974,5427.51532,3676.08875,3676.08875,722.67129,722.67129,722.67129,722.67129,722.67129,722.67129,0.00000,0.00000,0.00000,4398.76003


In [7]:
mse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'])
rmse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'], squared=False)
mad = metrics.mean_absolute_error(df_prophet['y'], forecast['yhat'])
mape = metrics.mean_absolute_percentage_error(df_prophet['y'], forecast['yhat'])

print(f'''
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      EQM: 764752.27
      REQM: 874.5
      MAD: 696.18
      MAPE: 10.57%
      


In [8]:
fig = go.Figure(data=[
    go.Scatter(name='Previsto',x=forecast['ds'], y=forecast['yhat'], marker={'color': 'orange'}),
    go.Scatter(name='Real', x=df_prophet['ds'], y=df_prophet['y'], marker={'color': 'gray'})
    ])
fig.update_layout(font=dict(color='black'), title_text='Real vs. Previsto (Prophet)')
fig.show()

### Tuning

In [9]:
param_grid = {  
    'changepoint_prior_scale': [0.8, 1.0],
    'seasonality_prior_scale': [0.8, 0.9, 1.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

In [10]:
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

In [11]:
mape = []

for params in all_params:
    m = Prophet(**params).fit(df_prophet) 
    df_cv = cross_validation(m, initial='1460 days', horizon='60 days', period='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    mape.append(df_p['mape'].values[0])

15:51:45 - cmdstanpy - INFO - Chain [1] start processing
15:51:46 - cmdstanpy - INFO - Chain [1] done processing
15:51:48 - cmdstanpy - INFO - Chain [1] start processing
15:51:48 - cmdstanpy - INFO - Chain [1] start processing
15:51:49 - cmdstanpy - INFO - Chain [1] done processing
15:51:49 - cmdstanpy - INFO - Chain [1] done processing
15:51:49 - cmdstanpy - INFO - deleting tmpfiles dir: /var/folders/zm/3bssg1p14lj63wq1qny6y2qh0000gn/T/tmp0hapl9xo
15:51:49 - cmdstanpy - INFO - deleting tmpfiles dir: /var/folders/zm/3bssg1p14lj63wq1qny6y2qh0000gn/T/tmpue20r2k7
15:51:49 - cmdstanpy - INFO - done
15:51:49 - cmdstanpy - INFO - done
15:51:49 - cmdstanpy - INFO - Chain [1] start processing
15:51:50 - cmdstanpy - INFO - Chain [1] done processing
15:51:52 - cmdstanpy - INFO - Chain [1] start processing
15:51:52 - cmdstanpy - INFO - Chain [1] start processing
15:51:53 - cmdstanpy - INFO - Chain [1] done processing
15:51:53 - cmdstanpy - INFO - Chain [1] done processing
15:51:53 - cmdstanpy - I

In [12]:
tuning_results = pd.DataFrame(all_params)
tuning_results['mape'] = mape
tuning_results

,changepoint_prior_scale,seasonality_prior_scale,seasonality_mode,mape
0,0.80000,0.80000,additive,0.08026
1,0.80000,0.80000,multiplicative,0.35809
2,0.80000,0.90000,additive,0.08044
3,0.80000,0.90000,multiplicative,0.34792
4,0.80000,1.00000,additive,0.08053
5,0.80000,1.00000,multiplicative,0.36012
6,1.00000,0.80000,additive,0.03691
7,1.00000,0.80000,multiplicative,0.24477
8,1.00000,0.90000,additive,0.03796
9,1.00000,0.90000,multiplicative,0.23989


In [13]:
best_params = all_params[np.argmin(mape)]
print(np.min(mape))
print(best_params)

0.03691367520181106
{'changepoint_prior_scale': 1.0, 'seasonality_prior_scale': 0.8, 'seasonality_mode': 'additive'}


In [14]:
model = Prophet(
    seasonality_mode=best_params['seasonality_mode'],
    seasonality_prior_scale=best_params['seasonality_prior_scale'],
    changepoint_prior_scale=best_params['changepoint_prior_scale'])

In [15]:
model.fit(df_prophet)

15:52:32 - cmdstanpy - INFO - Chain [1] start processing
15:52:32 - cmdstanpy - INFO - Chain [1] done processing


### Performance

In [16]:
forecast = model.predict(df_prophet)

mse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'])
rmse = metrics.mean_squared_error(df_prophet['y'], forecast['yhat'], squared=False)
mad = metrics.mean_absolute_error(df_prophet['y'], forecast['yhat'])
mape = metrics.mean_absolute_percentage_error(df_prophet['y'], forecast['yhat'])

print(f'''
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      EQM: 47189.23
      REQM: 217.23
      MAD: 160.91
      MAPE: 2.76%
      


In [17]:
fig = go.Figure(data=[
    go.Scatter(name='Previsto',x=forecast['ds'], y=forecast['yhat'], marker={'color': 'orange'}),
    go.Scatter(name='Real', x=df_prophet['ds'], y=df_prophet['y'], marker={'color': 'gray'})
    ])
fig.update_layout(font=dict(color='black'), title_text='Real vs. Previsto (Prophet)')
fig.show()

---
# Regressões

## Feature Engineering

### Criando LAGs

In [18]:
df_r = df.copy().drop(['vendas_Outra_Faculdade', 'vendas_Outra_Cidade', 'trend'], axis=1)
df_r.dropna(axis=0, inplace=True)
df_r.head()

,data,vendas,vendas_PUCRIO,vendas_UFABC,vendas_UFF,vendas_UFMG,vendas_UFPE,vendas_UFRJ,vendas_UFSC,vendas_UNICAMP,...,vendas_Natal,vendas_Niteroi,vendas_Porto_Alegre,vendas_Recife,vendas_Rio_de_Janeiro,vendas_Salvador,vendas_Santo_Andre,vendas_Sao_Carlos,vendas_Sao_Paulo,seasonal
0,2018-01-01,1819.00000,100.00000,17.00000,194.00000,34.00000,71.00000,407.00000,23.00000,200.00000,...,3.00000,79.00000,11.00000,51.00000,521.00000,25.00000,9.00000,14.00000,187.00000,-2401.78443
1,2018-02-01,2012.00000,98.00000,22.00000,199.00000,34.00000,85.00000,492.00000,28.00000,219.00000,...,2.00000,81.00000,11.00000,62.00000,592.00000,23.00000,13.00000,14.00000,205.00000,-1835.22888
2,2018-03-01,4035.00000,229.00000,16.00000,357.00000,46.00000,163.00000,1473.00000,30.00000,577.00000,...,9.00000,171.00000,14.00000,123.00000,1544.00000,22.00000,12.00000,29.00000,282.00000,935.86834
3,2018-04-01,4305.00000,189.00000,186.00000,512.00000,96.00000,134.00000,1098.00000,151.00000,434.00000,...,4.00000,232.00000,21.00000,105.00000,1300.00000,38.00000,40.00000,27.00000,359.00000,670.79890
4,2018-05-01,3956.00000,176.00000,130.00000,349.00000,212.00000,158.00000,721.00000,122.00000,376.00000,...,8.00000,155.00000,20.00000,110.00000,1065.00000,41.00000,39.00000,31.00000,300.00000,267.66001


In [19]:
lag_columns = ['vendas_PUCRIO', 'vendas_UFABC', 'vendas_UFF', 'vendas_UFMG', 'vendas_UFPE', 'vendas_UFRJ', 'vendas_UFSC', 'vendas_UNICAMP', 'vendas_USP', 'vendas_UTFPR', 'vendas_Belo_Horizonte',
       'vendas_Brasilia', 'vendas_Campinas', 'vendas_Campo_Grande',
       'vendas_Curitiba', 'vendas_Florianopolis', 'vendas_Fortaleza',
       'vendas_Goiania', 'vendas_Joinville', 'vendas_Juiz_de_Fora',
       'vendas_Manaus', 'vendas_Natal', 'vendas_Niteroi',
       'vendas_Porto_Alegre', 'vendas_Recife', 'vendas_Rio_de_Janeiro',
       'vendas_Salvador', 'vendas_Santo_Andre', 'vendas_Sao_Carlos',
       'vendas_Sao_Paulo']

In [20]:
for column in lag_columns:
    for lag in range(1, 4):
        novo_nome = f'{column}_LAG_{lag}'
        df_r[novo_nome] = df_r[column].shift(periods=lag, fill_value=0)

In [21]:
df_r['seasonal_LAG_12'] = df_r['seasonal'].shift(periods=12, fill_value=0)
df_r['vendas_LAG_12'] = df_r['vendas'].shift(periods=12, fill_value=0)

In [22]:
df_r.drop(lag_columns, axis=1, inplace=True)
df_r.drop('seasonal', axis=1, inplace=True)
df_r.head()

,data,vendas,vendas_PUCRIO_LAG_1,vendas_PUCRIO_LAG_2,vendas_PUCRIO_LAG_3,vendas_UFABC_LAG_1,vendas_UFABC_LAG_2,vendas_UFABC_LAG_3,vendas_UFF_LAG_1,vendas_UFF_LAG_2,...,vendas_Santo_Andre_LAG_2,vendas_Santo_Andre_LAG_3,vendas_Sao_Carlos_LAG_1,vendas_Sao_Carlos_LAG_2,vendas_Sao_Carlos_LAG_3,vendas_Sao_Paulo_LAG_1,vendas_Sao_Paulo_LAG_2,vendas_Sao_Paulo_LAG_3,seasonal_LAG_12,vendas_LAG_12
0,2018-01-01,1819.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,2018-02-01,2012.00000,100.00000,0.00000,0.00000,17.00000,0.00000,0.00000,194.00000,0.00000,...,0.00000,0.00000,14.00000,0.00000,0.00000,187.00000,0.00000,0.00000,0.00000,0.00000
2,2018-03-01,4035.00000,98.00000,100.00000,0.00000,22.00000,17.00000,0.00000,199.00000,194.00000,...,9.00000,0.00000,14.00000,14.00000,0.00000,205.00000,187.00000,0.00000,0.00000,0.00000
3,2018-04-01,4305.00000,229.00000,98.00000,100.00000,16.00000,22.00000,17.00000,357.00000,199.00000,...,13.00000,9.00000,29.00000,14.00000,14.00000,282.00000,205.00000,187.00000,0.00000,0.00000
4,2018-05-01,3956.00000,189.00000,229.00000,98.00000,186.00000,16.00000,22.00000,512.00000,357.00000,...,12.00000,13.00000,27.00000,29.00000,14.00000,359.00000,282.00000,205.00000,0.00000,0.00000


### Divisão entre Treino e Teste

In [23]:
X_r = df_r.drop(['data', 'vendas'], axis=1)
y_r = df_r['vendas']

In [24]:
X_r_treino, X_r_teste, y_r_treino, y_r_teste = train_test_split(X_r, y_r, train_size=0.65, random_state=123)

## Modelos

In [25]:
modelos = [LinearRegression(), Ridge(), Lasso(), DecisionTreeRegressor(), SVR(), AdaBoostRegressor(), ExtraTreesRegressor(), GradientBoostingRegressor(), RandomForestRegressor(), HistGradientBoostingRegressor(), LGBMRegressor(), CatBoostRegressor(verbose=False)]

In [26]:
modelos_score = []

for modelo in modelos:
    cv = cross_val_score(modelo, X_r_treino, y_r_treino, scoring='neg_mean_absolute_percentage_error')
    modelos_score.append({'modelo': modelo, 'mape': cv.mean()*(-100), 'std': cv.std()*(-100)})

In [27]:
modelos_score = pd.DataFrame(modelos_score)
modelos_score.sort_values(by='mape', ascending=True, inplace=True)
modelos_score

,modelo,mape,std
1,Ridge(),9.24343,-2.41849
0,LinearRegression(),9.24356,-2.41840
6,ExtraTreesRegressor(),14.52940,-3.90713
5,AdaBoostRegressor(),15.05235,-5.14623
8,RandomForestRegressor(),15.81873,-5.52248
11,<catboost.core.CatBoostRegressor object at 0x7...,16.04957,-4.63749
7,GradientBoostingRegressor(),16.65756,-4.79853
2,Lasso(),16.93542,-8.17960
3,DecisionTreeRegressor(),19.77880,-5.39478
9,HistGradientBoostingRegressor(),35.51489,-12.58749


### Linear Regression

In [28]:
linreg = LinearRegression()

linreg.fit(X_r_treino, y_r_treino)

LinearRegression()

##### Teste

In [29]:
linreg_pred_teste = linreg.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, linreg_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, linreg_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, linreg_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, linreg_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, linreg_pred_teste)

print(f'''
      Linear Regression - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Linear Regression - Teste
      R2: 0.66
      EQM: 2199636.55
      REQM: 1483.12
      MAD: 1053.04
      MAPE: 32.27%
      


##### Série Completa

In [30]:
linreg_pred = linreg.predict(X_r)

r2 = metrics.r2_score(y_r, linreg_pred)
mse = metrics.mean_squared_error(y_r, linreg_pred)
rmse = metrics.mean_squared_error(y_r, linreg_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, linreg_pred)
mape = metrics.mean_absolute_percentage_error(y_r, linreg_pred)

print(f'''
      Linear Regression - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Linear Regression - Série Completa
      R2: 0.91
      EQM: 803713.35
      REQM: 896.5
      MAD: 384.76
      MAPE: 11.79%
      


### Ridge

In [31]:
ridge = Ridge()

ridge.fit(X_r_treino, y_r_treino)

Ridge()

##### Teste

In [32]:
ridge_pred_teste = ridge.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, ridge_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, ridge_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, ridge_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, ridge_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, ridge_pred_teste)

print(f'''
      Ridge Regression - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Ridge Regression - Teste
      R2: 0.66
      EQM: 2199509.41
      REQM: 1483.07
      MAD: 1053.02
      MAPE: 32.27%
      


##### Série Completa

In [33]:
ridge_pred = ridge.predict(X_r)

r2 = metrics.r2_score(y_r, ridge_pred)
mse = metrics.mean_squared_error(y_r, ridge_pred)
rmse = metrics.mean_squared_error(y_r, ridge_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, ridge_pred)
mape = metrics.mean_absolute_percentage_error(y_r, ridge_pred)

print(f'''
      Ridge Regression - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Ridge Regression - Série Completa
      R2: 0.91
      EQM: 803666.9
      REQM: 896.47
      MAD: 384.77
      MAPE: 11.79%
      


### Decision Tree Regressor

In [34]:
dtreg = DecisionTreeRegressor()

dtreg.fit(X_r_treino, y_r_treino)

DecisionTreeRegressor()

##### Teste

In [35]:
dtreg_pred_teste = dtreg.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, dtreg_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, dtreg_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, dtreg_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, dtreg_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, dtreg_pred_teste)

print(f'''
      Decision Tree Regressor - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Decision Tree Regressor - Teste
      R2: 0.5
      EQM: 3243531.53
      REQM: 1800.98
      MAD: 1586.79
      MAPE: 40.75%
      


##### Série Completa

In [36]:
dtreg_pred = dtreg.predict(X_r)

r2 = metrics.r2_score(y_r, dtreg_pred)
mse = metrics.mean_squared_error(y_r, dtreg_pred)
rmse = metrics.mean_squared_error(y_r, dtreg_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, dtreg_pred)
mape = metrics.mean_absolute_percentage_error(y_r, dtreg_pred)

print(f'''
      Decision Tree Regressor - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Decision Tree Regressor - Série Completa
      R2: 0.86
      EQM: 1185136.52
      REQM: 1088.64
      MAD: 579.79
      MAPE: 14.89%
      


### SVR

In [37]:
svr = SVR()

svr.fit(X_r_treino, y_r_treino)

SVR()

##### Teste

In [38]:
svr_pred_teste = svr.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, svr_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, svr_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, svr_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, svr_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, svr_pred_teste)

print(f'''
      SVR - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      SVR - Teste
      R2: -0.94
      EQM: 12687724.8
      REQM: 3561.98
      MAD: 3018.04
      MAPE: 91.69%
      


##### Série Completa

In [39]:
svr_pred = svr.predict(X_r)

r2 = metrics.r2_score(y_r, svr_pred)
mse = metrics.mean_squared_error(y_r, svr_pred)
rmse = metrics.mean_squared_error(y_r, svr_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, svr_pred)
mape = metrics.mean_absolute_percentage_error(y_r, svr_pred)

print(f'''
      SVR - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      SVR - Série Completa
      R2: -0.06
      EQM: 9145401.07
      REQM: 3024.14
      MAD: 2608.6
      MAPE: 54.73%
      


### AdaBoost

In [40]:
adaboost = AdaBoostRegressor()

adaboost.fit(X_r_treino, y_r_treino)

AdaBoostRegressor()

##### Teste

In [41]:
adaboost_pred_teste = adaboost.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, adaboost_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, adaboost_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, adaboost_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, adaboost_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, adaboost_pred_teste)

print(f'''
      AdaBoost - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      AdaBoost - Teste
      R2: 0.4
      EQM: 3936051.97
      REQM: 1983.95
      MAD: 1829.2
      MAPE: 45.33%
      


##### Série Completa

In [42]:
adaboost_pred = adaboost.predict(X_r)

r2 = metrics.r2_score(y_r, adaboost_pred)
mse = metrics.mean_squared_error(y_r, adaboost_pred)
rmse = metrics.mean_squared_error(y_r, adaboost_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, adaboost_pred)
mape = metrics.mean_absolute_percentage_error(y_r, adaboost_pred)

print(f'''
      AdaBoost - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      AdaBoost - Série Completa
      R2: 0.83
      EQM: 1461254.42
      REQM: 1208.82
      MAD: 756.27
      MAPE: 17.8%
      


### Extra Trees Regressor

In [43]:
et_reg = ExtraTreesRegressor()

et_reg.fit(X_r_treino, y_r_treino)

ExtraTreesRegressor()

##### Teste

In [44]:
et_reg_pred_teste = et_reg.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, et_reg_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, et_reg_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, et_reg_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, et_reg_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, et_reg_pred_teste)

print(f'''
      Extra Trees - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Extra Trees - Teste
      R2: 0.46
      EQM: 3551082.0
      REQM: 1884.43
      MAD: 1637.25
      MAPE: 42.98%
      


##### Série Completa

In [45]:
et_reg_pred = et_reg.predict(X_r)

r2 = metrics.r2_score(y_r, et_reg_pred)
mse = metrics.mean_squared_error(y_r, et_reg_pred)
rmse = metrics.mean_squared_error(y_r, et_reg_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, et_reg_pred)
mape = metrics.mean_absolute_percentage_error(y_r, et_reg_pred)

print(f'''
      Extra Trees - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Extra Trees - Série Completa
      R2: 0.85
      EQM: 1297510.73
      REQM: 1139.08
      MAD: 598.23
      MAPE: 15.71%
      


### Gradient Boosting Regressor

In [46]:
gb_reg = GradientBoostingRegressor()

gb_reg.fit(X_r_treino, y_r_treino)

GradientBoostingRegressor()

##### Teste

In [47]:
gb_reg_pred_teste = gb_reg.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, gb_reg_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, gb_reg_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, gb_reg_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, gb_reg_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, gb_reg_pred_teste)

print(f'''
      Gradient Boosting - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Gradient Boosting - Teste
      R2: 0.51
      EQM: 3220478.48
      REQM: 1794.57
      MAD: 1620.31
      MAPE: 39.9%
      


##### Série Completa

In [48]:
gb_reg_pred = gb_reg.predict(X_r)

r2 = metrics.r2_score(y_r, gb_reg_pred)
mse = metrics.mean_squared_error(y_r, gb_reg_pred)
rmse = metrics.mean_squared_error(y_r, gb_reg_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, gb_reg_pred)
mape = metrics.mean_absolute_percentage_error(y_r, gb_reg_pred)

print(f'''
      Gradient Boosting - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Gradient Boosting - Série Completa
      R2: 0.86
      EQM: 1176713.75
      REQM: 1084.76
      MAD: 592.49
      MAPE: 14.58%
      


### Random Forest Regressor

In [49]:
rf_reg = RandomForestRegressor()

rf_reg.fit(X_r_treino, y_r_treino)

RandomForestRegressor()

##### Teste

In [50]:
rf_reg_pred_teste = rf_reg.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, rf_reg_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, rf_reg_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, rf_reg_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, rf_reg_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, rf_reg_pred_teste)

print(f'''
      Random Forest - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Random Forest - Teste
      R2: 0.43
      EQM: 3748663.5
      REQM: 1936.15
      MAD: 1756.28
      MAPE: 45.42%
      


##### Série Completa

In [51]:
rf_reg_pred = rf_reg.predict(X_r)

r2 = metrics.r2_score(y_r, rf_reg_pred)
mse = metrics.mean_squared_error(y_r, rf_reg_pred)
rmse = metrics.mean_squared_error(y_r, rf_reg_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, rf_reg_pred)
mape = metrics.mean_absolute_percentage_error(y_r, rf_reg_pred)

print(f'''
      Random Forest - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Random Forest - Série Completa
      R2: 0.83
      EQM: 1506265.81
      REQM: 1227.3
      MAD: 883.94
      MAPE: 19.81%
      


### Histocastic Gradient Boosting

In [52]:
histgrad = HistGradientBoostingRegressor()

histgrad.fit(X_r_treino, y_r_treino)

HistGradientBoostingRegressor()

##### Teste

In [53]:
histgrad_pred_teste = histgrad.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, histgrad_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, histgrad_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, histgrad_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, histgrad_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, histgrad_pred_teste)

print(f'''
      Histocastic Gradient Boosting - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Histocastic Gradient Boosting - Teste
      R2: -1.19
      EQM: 14318693.29
      REQM: 3784.0
      MAD: 3231.58
      MAPE: 97.97%
      


##### Série Completa

In [54]:
histgrad_pred = histgrad.predict(X_r)

r2 = metrics.r2_score(y_r, histgrad_pred)
mse = metrics.mean_squared_error(y_r, histgrad_pred)
rmse = metrics.mean_squared_error(y_r, histgrad_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, histgrad_pred)
mape = metrics.mean_absolute_percentage_error(y_r, histgrad_pred)

print(f'''
      Histocastic Gradient Boosting - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Histocastic Gradient Boosting - Série Completa
      R2: -0.12
      EQM: 9697360.69
      REQM: 3114.06
      MAD: 2694.91
      MAPE: 57.95%
      


### LGBM

In [55]:
lgbm = LGBMRegressor()

lgbm.fit(X_r_treino, y_r_treino)

LGBMRegressor()

##### Teste

In [56]:
lgbm_pred_teste = lgbm.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, lgbm_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, lgbm_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, lgbm_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, lgbm_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, lgbm_pred_teste)

print(f'''
      LGBM - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      LGBM - Teste
      R2: -1.19
      EQM: 14318693.29
      REQM: 3784.0
      MAD: 3231.58
      MAPE: 97.97%
      


##### Série Completa

In [57]:
lgbm_pred = lgbm.predict(X_r)

r2 = metrics.r2_score(y_r, lgbm_pred)
mse = metrics.mean_squared_error(y_r, lgbm_pred)
rmse = metrics.mean_squared_error(y_r, lgbm_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, lgbm_pred)
mape = metrics.mean_absolute_percentage_error(y_r, lgbm_pred)

print(f'''
      LGBM - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      LGBM - Série Completa
      R2: -0.12
      EQM: 9697360.69
      REQM: 3114.06
      MAD: 2694.91
      MAPE: 57.95%
      


### CatBoost

In [58]:
catboost = CatBoostRegressor(verbose=False)

catboost.fit(X_r_treino, y_r_treino)

##### Teste

In [59]:
catboost_pred_teste = catboost.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, catboost_pred_teste)
mse = metrics.mean_squared_error(y_r_teste, catboost_pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, catboost_pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, catboost_pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, catboost_pred_teste)

print(f'''
      CatBoost - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      CatBoost - Teste
      R2: 0.4
      EQM: 3923827.49
      REQM: 1980.87
      MAD: 1767.14
      MAPE: 46.29%
      


##### Série Completa

In [60]:
catboost_pred = catboost.predict(X_r)

r2 = metrics.r2_score(y_r, catboost_pred)
mse = metrics.mean_squared_error(y_r, catboost_pred)
rmse = metrics.mean_squared_error(y_r, catboost_pred, squared=False)
mad = metrics.mean_absolute_error(y_r, catboost_pred)
mape = metrics.mean_absolute_percentage_error(y_r, catboost_pred)

print(f'''
      CatBoost - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      CatBoost - Série Completa
      R2: 0.83
      EQM: 1433708.15
      REQM: 1197.38
      MAD: 646.58
      MAPE: 16.93%
      


# Ridge

In [61]:
pipe = Pipeline(steps=[
    ('preprocessor', StandardScaler()),
    ('model', Ridge())
])

In [62]:
pipe.fit(X_r_treino, y_r_treino)

Pipeline(steps=[('preprocessor', StandardScaler()), ('model', Ridge())])

In [63]:
pred_teste = pipe.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, pred_teste)
mse = metrics.mean_squared_error(y_r_teste, pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, pred_teste)

print(f'''
      Ridge Regression Standard - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Ridge Regression Standard - Teste
      R2: 0.74
      EQM: 1728008.93
      REQM: 1314.54
      MAD: 960.39
      MAPE: 28.66%
      


### Tuning

In [64]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=123)

In [65]:
grid = dict(model__alpha=np.linspace(1, 10, 21))
grid

{'model__alpha': array([ 1.  ,  1.45,  1.9 ,  2.35,  2.8 ,  3.25,  3.7 ,  4.15,  4.6 ,
         5.05,  5.5 ,  5.95,  6.4 ,  6.85,  7.3 ,  7.75,  8.2 ,  8.65,
         9.1 ,  9.55, 10.  ])}

In [66]:
gscv = GridSearchCV(pipe, grid, scoring='neg_mean_absolute_percentage_error', n_jobs=1, cv=cv)
gscv.fit(X_r_treino, y_r_treino)

GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=123),
             estimator=Pipeline(steps=[('preprocessor', StandardScaler()),
                                       ('model', Ridge())]),
             n_jobs=1,
             param_grid={'model__alpha': array([ 1.  ,  1.45,  1.9 ,  2.35,  2.8 ,  3.25,  3.7 ,  4.15,  4.6 ,
        5.05,  5.5 ,  5.95,  6.4 ,  6.85,  7.3 ,  7.75,  8.2 ,  8.65,
        9.1 ,  9.55, 10.  ])},
             scoring='neg_mean_absolute_percentage_error')

In [67]:
gscv.best_params_

{'model__alpha': 1.0}

In [68]:
pipe_tuned = Pipeline(steps=[
    ('preprocessor', StandardScaler()),
    ('model', Ridge(alpha=5.5))
])

In [69]:
pipe_tuned.fit(X_r_treino, y_r_treino)

Pipeline(steps=[('preprocessor', StandardScaler()),
                ('model', Ridge(alpha=5.5))])

In [70]:
pred_teste = pipe_tuned.predict(X_r_teste)

r2 = metrics.r2_score(y_r_teste, pred_teste)
mse = metrics.mean_squared_error(y_r_teste, pred_teste)
rmse = metrics.mean_squared_error(y_r_teste, pred_teste, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, pred_teste)
mape = metrics.mean_absolute_percentage_error(y_r_teste, pred_teste)

print(f'''
      Ridge Regression Standard Tuned - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Ridge Regression Standard Tuned - Teste
      R2: 0.73
      EQM: 1765652.92
      REQM: 1328.78
      MAD: 1073.55
      MAPE: 30.82%
      


### Feature Selection

In [71]:
bfs = SequentialFeatureSelector(pipe_tuned, k_features='best', forward=False, n_jobs=1, cv=cv)

In [72]:
bfs.fit(X_r_treino, y_r_treino)

SequentialFeatureSelector(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=123),
                          estimator=Pipeline(steps=[('preprocessor',
                                                     StandardScaler()),
                                                    ('model',
                                                     Ridge(alpha=5.5))]),
                          forward=False, k_features='best')

In [73]:
bfs_features = list(bfs.k_feature_names_)
bfs_features

['vendas_PUCRIO_LAG_2',
 'vendas_PUCRIO_LAG_3',
 'vendas_UFABC_LAG_2',
 'vendas_UFF_LAG_2',
 'vendas_UFMG_LAG_1',
 'vendas_UFRJ_LAG_1',
 'vendas_UFSC_LAG_1',
 'vendas_UNICAMP_LAG_1',
 'vendas_UNICAMP_LAG_3',
 'vendas_UTFPR_LAG_1',
 'vendas_Belo_Horizonte_LAG_1',
 'vendas_Campinas_LAG_1',
 'vendas_Campo_Grande_LAG_3',
 'vendas_Curitiba_LAG_3',
 'vendas_Florianopolis_LAG_2',
 'vendas_Goiania_LAG_3',
 'vendas_Joinville_LAG_2',
 'vendas_Manaus_LAG_2',
 'vendas_Manaus_LAG_3',
 'vendas_Natal_LAG_3',
 'vendas_Niteroi_LAG_2',
 'vendas_Recife_LAG_3',
 'vendas_Salvador_LAG_2',
 'vendas_Santo_Andre_LAG_1',
 'vendas_Sao_Carlos_LAG_3',
 'seasonal_LAG_12']

In [74]:
X_r_treino_bfs = X_r_treino.loc[:, bfs_features]
X_r_teste_bfs = X_r_teste.loc[:, bfs_features]
X_r_bfs = X_r.loc[:, bfs_features]

In [75]:
pipe_tuned_bfs = Pipeline(steps=[
    ('preprocessor', StandardScaler()),
    ('model', Ridge(alpha=5.5))
])

In [76]:
pipe_tuned_bfs.fit(X_r_treino_bfs, y_r_treino)

Pipeline(steps=[('preprocessor', StandardScaler()),
                ('model', Ridge(alpha=5.5))])

In [77]:
pred_teste_bfs = pipe_tuned_bfs.predict(X_r_teste_bfs)

r2 = metrics.r2_score(y_r_teste, pred_teste_bfs)
mse = metrics.mean_squared_error(y_r_teste, pred_teste_bfs)
rmse = metrics.mean_squared_error(y_r_teste, pred_teste_bfs, squared=False)
mad = metrics.mean_absolute_error(y_r_teste, pred_teste_bfs)
mape = metrics.mean_absolute_percentage_error(y_r_teste, pred_teste_bfs)

print(f'''
      Ridge Regression Standard Tuned - Teste
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Ridge Regression Standard Tuned - Teste
      R2: 0.71
      EQM: 1882527.91
      REQM: 1372.05
      MAD: 1055.05
      MAPE: 30.93%
      


In [78]:
pred_bfs = pipe_tuned_bfs.predict(X_r_bfs)

r2 = metrics.r2_score(y_r, pred_bfs)
mse = metrics.mean_squared_error(y_r, pred_bfs)
rmse = metrics.mean_squared_error(y_r, pred_bfs, squared=False)
mad = metrics.mean_absolute_error(y_r, pred_bfs)
mape = metrics.mean_absolute_percentage_error(y_r, pred_bfs)

print(f'''
      Ridge Regression Standard Tuned - Série Completa
      R2: {round(r2,2)}
      EQM: {round(mse,2)}
      REQM: {round(rmse,2)}
      MAD: {round(mad,2)}
      MAPE: {round(mape*100,2)}%
      ''')


      Ridge Regression Standard Tuned - Série Completa
      R2: 0.91
      EQM: 752418.48
      REQM: 867.42
      MAD: 547.13
      MAPE: 13.47%
      


In [79]:
fig = go.Figure(data=[
    go.Scatter(name='Previsto',x=df_r['data'], y=pred_bfs, marker={'color': 'orange'}),
    go.Scatter(name='Real', x=df_r['data'], y=y_r, marker={'color': 'gray'})
    ])
fig.update_layout(font=dict(color='black'), title_text='Real vs. Previsto (Ridge Regression)')
fig.show()